# Credit Risk Resampling Techniques

In [44]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [48]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [49]:
# Create our features
X =  df.copy()
X.drop(["loan_status"], axis=1, inplace=True)
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']


In [50]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [51]:
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0


In [53]:
# Check the balance of our target values
df['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [54]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [55]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [56]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [57]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [59]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [60]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [61]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    0,   619],
       [    0, 18765]])

In [62]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00       619
   low_risk       0.97      1.00      0.00      0.98      0.00      0.00     18765

avg / total       0.94      0.97      0.03      0.95      0.00      0.00     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [63]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [64]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [66]:
y_pred_os = model.predict(X_test_scaled)

In [67]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred_os)

0.7946990388303251

In [68]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_os)

array([[  616,     3],
       [ 7614, 11151]])

In [69]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_os))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      1.00      0.59      0.14      0.77      0.62       619
   low_risk       1.00      0.59      1.00      0.75      0.77      0.57     18765

avg / total       0.97      0.61      0.98      0.73      0.77      0.57     19384



### SMOTE Oversampling

In [70]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
# YOUR CODE HERE
from collections import Counter
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [71]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [72]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_SMOTE = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_SMOTE)

0.9936781215845847

In [73]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_SMOTE)

array([[  615,     4],
       [  116, 18649]])

In [74]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_SMOTE))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [75]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)


# View the count of target classes with Counter
# YOUR CODE HERE
from collections import Counter
Counter(y_resampled)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [76]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [81]:
y_pred_US = model.predict(X_test)

In [82]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred_US)

0.9865149130022852

In [83]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_US)

array([[  606,    13],
       [  112, 18653]])

In [84]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_US))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.97       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [85]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'high_risk': 55626, 'low_risk': 55948})

In [86]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [87]:
y_pred_OOS = model.predict(X_test)

In [88]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred_OOS)

0.9935182494822666

In [89]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_OOS)

array([[  615,     4],
       [  122, 18643]])

In [90]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_OOS))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE: SMOTE Oversampling model had the best balanced accuracy score (0.9937) which is the highest compare to models (Combination Sampling:0.9935; Undersampling: 0.987; Naive Random Oversampling: 0.795 ; Simple Logistic Regression: 0.5)
   
2. Which model had the best recall score?

    YOUR ANSWER HERE: Three models (Combination Sampling, Undersampling and SMOTE Oversampling) all have the same best recall score 0.99, that is the highest recall score. The other two models recall scores are
lower (Naive Random Oversampling:0.61 ; Simple Logistic Regression:0.97)


3. Which model had the best geometric mean score?

    YOUR ANSWER HERE:Three models (Combination Sampling, Undersampling and SMOTE Oversampling) all have the same best geometric mean score 0.99, that's the highest geometric mean score. The other two models
geometric mean scores are lower (Naive Random Oversampling:0.77 ; Simple Logistic Regression:0.00)